In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
data = pd.read_csv('2020.csv')
data.head()

C:\Users\xfuen\AppData\Local\Temp\ipykernel_17804\528515669.py:1: DtypeWarning: Columns (16,71,72,81,164,165,179,180) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2020.csv')


,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,...,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
0,6800_-NONE-_68HE0120P0027_0_-NONE-_0,CONT_AWD_68HE0120P0027_6800_-NONE-_-NONE-,68HE0120P0027,0,0.0,NaN,NaN,NaN,NaN,11950.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_68H...,2020-09-30,2020-09-30
1,6800_-NONE-_EPS71506_P00046_-NONE-_-NONE-,CONT_IDV_EPS71506_6800,EPS71506,P00046,NaN,NaN,NaN,NaN,NaN,0.00,...,195644.0,"RONALD E MCMANAMY, JR.",141166.0,GLEN H SHERRIER,140799.0,THOMAS C GERSTLE,133556.0,https://www.usaspending.gov/award/CONT_IDV_EPS...,2020-09-30,2020-09-30
2,6800_6800_68HE0320F0029_P00001_EPBPA17R30001_0,CONT_AWD_68HE0320F0029_6800_EPBPA17R30001_6800,68HE0320F0029,P00001,0.0,6800.0,ENVIRONMENTAL PROTECTION AGENCY,EPBPA17R30001,P00001,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_68H...,2020-09-30,2023-02-24
3,6800_6800_68HE0320F0051_P00006_68HE0320D0003_0,CONT_AWD_68HE0320F0051_6800_68HE0320D0003_6800,68HE0320F0051,P00006,0.0,6800.0,ENVIRONMENTAL PROTECTION AGENCY,68HE0320D0003,0,80000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_68H...,2020-09-30,2020-09-30
4,6800_4730_EPG14H00832_P00033_GS35F0585J_0,CONT_AWD_EPG14H00832_6800_GS35F0585J_4730,EPG14H00832,P00033,0.0,4730.0,FEDERAL ACQUISITION SERVICE,GS35F0585J,PVA529,-61877.59,...,445672.0,BROCK LENDING,461333.0,MARK TICE,421358.0,PAUL EDELMAN,277435.0,https://www.usaspending.gov/award/CONT_AWD_EPG...,2020-07-30,2020-09-30


In [5]:
# Data cleaning
data = data[~data['potential_total_value_of_award'].isnull()]  # Remove rows with missing potential total value
data['potential_total_value_of_award'] = data['potential_total_value_of_award'].astype(float)  # Convert to float

# Visualizations
fig = px.histogram(data, x='potential_total_value_of_award', title='Distribution of Potential Total Value of EPA Awards', nbins=50)
fig.update_xaxes(title='Potential Total Value (USD)')
fig.update_yaxes(title='Count')
fig.show()

fig = px.box(data, x='woman_owned_business', y='potential_total_value_of_award', title='Potential Total Value by Woman-Owned Business Status')
fig.update_xaxes(title='Woman-Owned Business')
fig.update_yaxes(title='Potential Total Value (USD)')
fig.show()

fig = px.box(data, x='minority_owned_business', y='potential_total_value_of_award', title='Potential Total Value by Minority-Owned Business Status')
fig.update_xaxes(title='Minority-Owned Business')
fig.update_yaxes(title='Potential Total Value (USD)')
fig.show()

In [6]:
# Funding by business type
business_type_summary = data.groupby('organizational_type')['potential_total_value_of_award'].sum().reset_index()

# Create the pie chart
fig = px.pie(business_type_summary, values='potential_total_value_of_award', names='organizational_type',
             title='Potential Total Value by Organizational Type',
             labels={'organizational_type': 'Organizational Type', 'potential_total_value_of_award': 'Potential Total Value (USD)'})

fig.show()


In [7]:
# Funding by product/service code (top 10)
product_service_code_summary = data.groupby('product_or_service_code_description')['potential_total_value_of_award'].sum().sort_values(ascending=False).head(10).reset_index()

# Create the pie chart
fig = px.pie(product_service_code_summary, values='potential_total_value_of_award', names='product_or_service_code_description',
             title='Potential Total Value by Product/Service Code (Top 10)',
             labels={'product_or_service_code_description': 'Product/Service Code Description', 'potential_total_value_of_award': 'Potential Total Value (USD)'})

fig.show()


In [8]:
# Calculate the mean award amount by NAICS description
mean_award_by_naics = data.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index()

# Sort the data by mean award amount and select the top 10
mean_award_by_naics = mean_award_by_naics.sort_values(by='potential_total_value_of_award', ascending=False).head(10)

# Create the pie chart
fig = px.pie(mean_award_by_naics, values='potential_total_value_of_award', names='naics_description',
             title='Mean Award Amount by Company Type (Top 10)',
             labels={'naics_description': 'Company Type (NAICS Description)', 'potential_total_value_of_award': 'Mean Award Amount (USD)'})

fig.show()


In [9]:
# Women-owned businesses
women_owned = data[data['woman_owned_business'] == 't']
mean_award_women_owned = women_owned.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_women_owned, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Women-Owned Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()

# Minority-owned businesses
minority_owned = data[data['minority_owned_business'] == 't']
mean_award_minority_owned = minority_owned.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_minority_owned, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Minority-Owned Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()

# Small businesses
small_business = data[data['contracting_officers_determination_of_business_size_code'] == 'S']
mean_award_small_business = small_business.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_small_business, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Small Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()